# Process Model Outputs

In [1]:
# Imports
import os
import shutil

import cv2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cmcrameri.cm as cmc
import pyvista as pv

#pv.start_xvfb()

from tqdm import tqdm

import gdtchron as gdt

In [2]:
# Get files from model output
part_path = 'output_uplift_box_fixed/particles'
files = [os.path.join(part_path,file) for file in os.listdir(part_path) if file.endswith('.pvtu')]
files.sort()

In [4]:
gdt.run_vtk(files=files,system='AHe',time_interval=0.1,processes=10)

Timestep 16:   1%|          | 1200/98000 [11:44<01:04, 1505.85it/s]/opt/conda/envs/aspect-docker/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Timestep 200: 100%|██████████| 95979/95979 [00:19<00:00, 4887.16it/s]


In [ ]:
# Set path to reuse files from AHe
new_path = 'meshes_tchron_uplift'
files = [os.path.join(new_path,file) for file in os.listdir(new_path) if file.endswith('.vtu')]
files.sort()

In [ ]:
# Add AFT
calculate.tchron_vtk_parallel(files,'AFT',1e5,
                                model_inputs=(aft.Ketcham_99_FC, 1.75),
                                model_fn=calculate.particle_FT_annealing,
                                batch_size=100,
                                processes=os.cpu_count()/2,
                                overwrite=False,
                                file_prefix='meshes_tchron_uplift')

In [ ]:
# Add ZHe
calculate.tchron_vtk_parallel(files,'ZHe',1e5,
                                model_inputs=(100,100,50),
                                model_fn=calculate.particle_He_profile,
                                batch_size=100,
                                processes=os.cpu_count()/2,
                                overwrite=False,
                                file_prefix='meshes_tchron_uplift')

In [ ]:
# Set up video
directory = 'meshes_tchron_uplift'

time_total = 20 # Myr
model_step = 0.1 # Myr
bounds = [35,65,10,22] # km

nsteps = int(time_total/model_step)

timesteps = np.arange(0,nsteps+1,1)

files = [os.path.join(directory,file) for file in os.listdir(directory) if file.endswith('.vtu')]
files.sort()

image_dir = 'images/'

ahe_cmap = cmc.lapaz_r
aft_cmap = cmc.lajolla_r
zhe_cmap = cmc.bamako_r
cat_cmap = cmc.batlowS
clim = [0,time_total]
bar=True

In [ ]:
# Run image creation
try:
    shutil.rmtree(image_dir)
except:
    print("Creating new image directory...")
else:
    print("Cleared existing image directory...")

os.makedirs(image_dir,exist_ok=False)

# Make images for the video
for k,step in enumerate(tqdm(timesteps)):
    time = step*model_step
    time_str = str(round(step/2,1)).zfill(5).replace('.','-')
    
    fig,axs = plt.subplots(2,2,dpi=300,figsize=(6.5,6))
    axs = axs.flat

    if 10<=time<=15:
        title = 'Uplift'
    else:
        title = 'Quiescence'

    axs[3].set_title(str(round(time,1)) +' Ma - ' + title,loc='left')
    
    plot.plot2D(files[k],'AHe',bounds=bounds,ax=axs[0],
              cmap=ahe_cmap,colorbar=bar,clim=clim)
    
    plot.plot2D(files[k],'AFT',bounds=bounds,ax=axs[1],
            cmap=aft_cmap,colorbar=bar,clim=clim)
    
    plot.plot2D(files[k],'ZHe',bounds=bounds,ax=axs[2],
        cmap=zhe_cmap,colorbar=bar,clim=clim)

    axs[0].set_title('AHe')
    axs[1].set_title('AFT')
    axs[2].set_title('ZHe')
    
    mesh = pv.read(files[k])

    y = np.round(mesh.points[:,1]/1000,0)
    AHe = mesh['AHe']
    AFT = mesh['AFT']
    ZHe = mesh['ZHe']

    df = pd.DataFrame({'y':y,'AHe':AHe,'AFT':AFT,'ZHe':ZHe})
    df_max = df.groupby('y').agg({'y':'first','AHe':'max','AFT':'max','ZHe':'max'})

    axs[3].plot(df_max['AHe'],df_max['y'],c=cat_cmap.colors[6],label='AHe')
    axs[3].plot(df_max['AFT'],df_max['y'],c=cat_cmap.colors[4],label='AFT')
    axs[3].plot(df_max['ZHe'],df_max['y'],c=cat_cmap.colors[2],label='ZHe')

    axs[3].set_xlim(-0.25,time_total)
    axs[3].set_ylim(bounds[2],bounds[3])
    axs[3].set_xlabel('Maximum Age (Ma)',fontsize=6)
    axs[3].set_ylabel('Y Position (km)',fontsize=6)
    axs[3].tick_params(labelsize=6)
    axs[3].text(0.5,0.2,
                'Surface Ages:\n'
                  'AHe: ' + str(round(df_max['AHe'][20],1)) + ' Ma\n'
                  'AFT: ' + str(round(df_max['AFT'][20],1)) + ' Ma\n'
                  'ZHe: ' + str(round(df_max['ZHe'][20],1)) + ' Ma\n',
                transform=axs[3].transAxes)
    axs[3].legend(fontsize=6)

    for ax in [axs[0],axs[1],axs[2]]:
        ax.set_xlabel('X Position (km)',fontsize=6)
        ax.set_ylabel('Y Position (km)',fontsize=6)
        ax.tick_params(labelsize=6)

    plt.tight_layout()
    fig.savefig(image_dir+time_str+'.jpg')
    plt.close()

In [ ]:
# Make movie
img_paths = [image_dir+file for file in sorted(os.listdir(image_dir)) if file.endswith('.jpg')]

frame = cv2.imread(img_paths[0])
height,width,layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'avc1')

frate = 1/model_step

video = cv2.VideoWriter('video_tchron.mp4', fourcc, frate, (width,height))

for img in img_paths:
    video.write(cv2.imread(img))

cv2.destroyAllWindows()
video.release()

In [ ]:
from IPython.display import Video

video_path = 'video_tchron.mp4'
Video(video_path, embed=True, width=800)